In [29]:
from nupic.research.frameworks import vernon
from copy import copy, deepcopy
from pprint import pprint


In [30]:
# dir(vernon)

## Supervised Learning

In [31]:
import nupic.research
dir(nupic.research.frameworks.vernon)

['RezeroedKWinnersGSCExperiment',
 'SupervisedExperiment',
 'VariedRezeroedKWinnersGSCExperiment',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'common_experiments',
 'components',
 'experiment_utils',
 'experiments',
 'interfaces',
 'mixins',
 'network_utils',
 'supervised_experiment']

In [32]:
from nupic.research.frameworks.vernon import SupervisedExperiment

In [33]:
import torch 
from torchvision import datasets, transforms, models

from nupic.research.frameworks.vernon import SupervisedExperiment

supervised_test = dict(
    # dataset -  using torchvision
    dataset_class=datasets.CIFAR10,
    dataset_args=dict(root="~/nta/datasets", transform=transforms.ToTensor()),       # model - using torchvision
    model_class=models.resnet18,
    model_args=dict(num_classes=10, pretrained=False),
    num_classes=10,
    # experiment
    distributed=False,
    # hyperparameters
    batch_size=32,
    epochs=5,
    optimizer_args=dict(lr=1e-2),
    optimizer_class=torch.optim.Adam,
    # for debugging
    batches_in_epoch=10,
    batches_in_epoch_val=10,
)

def run(experiment_class, config):
    exp = experiment_class()
    exp.setup_experiment(config)

    epoch = 0
    while not exp.should_stop():
        epoch += 1
        results = exp.run_epoch()
        print(f"Epoch: {epoch} Acc: {results['mean_accuracy']:.2f}")
        print(results)

run(SupervisedExperiment, config=supervised_test)

INFO:SupervisedExperiment:Execution order: {'complexity_loss': [],
 'create_loaders': ['SupervisedExperiment.create_loaders'],
 'create_lr_scheduler': ['SupervisedExperiment.create_lr_scheduler'],
 'create_model': ['SupervisedExperiment.create_model'],
 'create_optimizer': ['SupervisedExperiment.create_optimizer'],
 'create_train_dataloader': ['SupervisedExperiment.create_train_dataloader'],
 'create_train_sampler': ['SupervisedExperiment.create_train_sampler'],
 'create_validation_dataloader': ['SupervisedExperiment.create_validation_dataloader'],
 'create_validation_sampler': ['SupervisedExperiment.create_validation_sampler'],
 'error_loss': ['SupervisedExperiment.error_loss'],
 'get_readable_result': ['SupervisedExperiment: Basic keys'],
 'get_state': ['SupervisedExperiment: Model, optimizer, LR scheduler, epoch'],
 'insert_pre_experiment_result': [],
 'load_dataset': ['SupervisedExperiment.load_dataset'],
 'post_batch': ['SupervisedExperiment: Logging'],
 'post_epoch': [],
 'post_o

In [34]:
from nupic.research.frameworks.vernon import mixins

class CutMixSupervisedExperiment(mixins.CutMix,
                                 SupervisedExperiment):
    pass

supervised_test_v2 = deepcopy(supervised_test)
supervised_test_v2.update(
    experiment_class=CutMixSupervisedExperiment,
    mixup_beta=1.0,
    cutmix_prob=0.8,
)

run(SupervisedExperiment, config=supervised_test_v2)

INFO:SupervisedExperiment:Execution order: {'complexity_loss': [],
 'create_loaders': ['SupervisedExperiment.create_loaders'],
 'create_lr_scheduler': ['SupervisedExperiment.create_lr_scheduler'],
 'create_model': ['SupervisedExperiment.create_model'],
 'create_optimizer': ['SupervisedExperiment.create_optimizer'],
 'create_train_dataloader': ['SupervisedExperiment.create_train_dataloader'],
 'create_train_sampler': ['SupervisedExperiment.create_train_sampler'],
 'create_validation_dataloader': ['SupervisedExperiment.create_validation_dataloader'],
 'create_validation_sampler': ['SupervisedExperiment.create_validation_sampler'],
 'error_loss': ['SupervisedExperiment.error_loss'],
 'get_readable_result': ['SupervisedExperiment: Basic keys'],
 'get_state': ['SupervisedExperiment: Model, optimizer, LR scheduler, epoch'],
 'insert_pre_experiment_result': [],
 'load_dataset': ['SupervisedExperiment.load_dataset'],
 'post_batch': ['SupervisedExperiment: Logging'],
 'post_epoch': [],
 'post_o

In [35]:
dir(mixins)

['CompositeLoss',
 'ConfigureOptimizerParamGroups',
 'ConstrainParameters',
 'CutMix',
 'CutMixKnowledgeDistillation',
 'DelayLoadCheckpoint',
 'ExportModel',
 'ExtraValidationsPerEpoch',
 'GradientMetrics',
 'KnowledgeDistillation',
 'KnowledgeDistillationCL',
 'LRRangeTest',
 'LegacyImagenetConfig',
 'LoadPreprocessedData',
 'LogBackpropStructure',
 'LogCovariance',
 'LogEveryLearningRate',
 'LogEveryLoss',
 'MaxupPerSample',
 'MaxupStandard',
 'MultiCycleLR',
 'NoiseRobustnessTest',
 'Profile',
 'ProfileAutograd',
 'PruneLowMagnitude',
 'PruneLowSNRGlobal',
 'PruneLowSNRLayers',
 'RegularizeLoss',
 'ReportMaxAccuracy',
 'RezeroWeights',
 'SaveFinalCheckpoint',
 'StepBasedLogging',
 'TorchProfilerMixin',
 'TrackRepresentationSparsity',
 'UpdateBoostStrength',
 'VaryBatchSize',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'composite_loss',
 'configure_optimizer_param_groups',
 'constrain_parameters',
 '

In [36]:
# models and datasets available - not part of Vernon
from nupic.research.frameworks.pytorch import models as local_models
from nupic.research.frameworks.pytorch import datasets as local_datasets

supervised_test_v3 = deepcopy(supervised_test)
supervised_test_v3.update(
    # dataset - alternative using torchvision factory
    # includes base transforms as transforming to tensor and normalization
    dataset_class=local_datasets.torchvisiondataset,
    dataset_args=dict(root="~/nta/datasets", dataset_name="CIFAR10"),
    # can use local models available
    model_class=local_models.resnet9,
    model_args=dict(num_classes=10),
)

run(SupervisedExperiment, config=supervised_test_v3)

INFO:SupervisedExperiment:Execution order: {'complexity_loss': [],
 'create_loaders': ['SupervisedExperiment.create_loaders'],
 'create_lr_scheduler': ['SupervisedExperiment.create_lr_scheduler'],
 'create_model': ['SupervisedExperiment.create_model'],
 'create_optimizer': ['SupervisedExperiment.create_optimizer'],
 'create_train_dataloader': ['SupervisedExperiment.create_train_dataloader'],
 'create_train_sampler': ['SupervisedExperiment.create_train_sampler'],
 'create_validation_dataloader': ['SupervisedExperiment.create_validation_dataloader'],
 'create_validation_sampler': ['SupervisedExperiment.create_validation_sampler'],
 'error_loss': ['SupervisedExperiment.error_loss'],
 'get_readable_result': ['SupervisedExperiment: Basic keys'],
 'get_state': ['SupervisedExperiment: Model, optimizer, LR scheduler, epoch'],
 'insert_pre_experiment_result': [],
 'load_dataset': ['SupervisedExperiment.load_dataset'],
 'post_batch': ['SupervisedExperiment: Logging'],
 'post_epoch': [],
 'post_o

In [37]:
dir(local_datasets)

['ContextDependentPermutedMNIST',
 'PermutedMNIST',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'download_gsc_data',
 'gsc_factory',
 'imagenet',
 'imagenet_factory',
 'omniglot',
 'permuted_mnist',
 'preprocessed_gsc',
 'torchvision_factory',
 'torchvisiondataset']

In [38]:
dir(local_models)

['DenseNetCIFAR',
 'LeSparseNet',
 'MobileNetV1',
 'ModifiedInitStandardMLP',
 'NoSoDenseNetCIFAR',
 'ResNet',
 'SparseMLP',
 'StandardMLP',
 'VGGSparseNet',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'common_models',
 'le_sparse_net',
 'mobile_net_v1_sparse_depth',
 'mobile_net_v1_sparse_point',
 'mobilenetv1',
 'not_so_densenet',
 'resnet9',
 'resnet_models',
 'resnets',
 'separable_convolution2d',
 'vgg19_dense_net',
 'vgg19_sparse_net',
 'vgg_sparse_net']

## Continual Learning

In [40]:
from nupic.research.frameworks.continual_learning.experiments import ContinualLearningExperiment
from nupic.research.frameworks.pytorch.models import StandardMLP
from nupic.research.frameworks.continual_learning.mixins import ReduceLRAfterTask
from nupic.research.frameworks.pytorch.datasets import torchvisiondataset

class ReduceLRContinualLearningExperiment(ReduceLRAfterTask,
                                          ContinualLearningExperiment):
    pass

cl_mnist = dict(
    # specific to continual learning
    distributed=False,
    experiment_class=ReduceLRContinualLearningExperiment,
    num_classes=10,
    num_tasks=5,
    evaluation_metrics=[
        "eval_current_task",
        "eval_all_visited_tasks",
    ],
    # dataset
    dataset_class=torchvisiondataset,
    dataset_args=dict(root="~/nta/datasets", dataset_name="MNIST"),    
    # regular experiments
    model_class=StandardMLP,
    model_args=dict(
        input_size=(28, 28), num_classes=10, hidden_sizes=(50, 50, 50)
    ),
    # hyperparameters
    epochs_to_validate=[],
    epochs=5,
    batch_size=32,
    optimizer_class=torch.optim.SGD,
    optimizer_args=dict(lr=0.01, momentum=0.9, nesterov=False),
    # for debugging
    batches_in_epoch=30,
    batches_in_epoch_val=30
)

from nupic.research.frameworks.ray.run_with_raytune import run_single_instance
run_single_instance(cl_mnist)


2021-12-13 15:52:01,091	INFO resource_spec.py:223 -- Starting Ray with 9.91 GiB memory available for workers and up to 4.97 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
{'checkpoint_at_end': False,
 'checkpoint_freq': 0,
 'checkpoint_score_attr': None,
 'config': {'batch_size': 32,
            'batches_in_epoch': 30,
            'batches_in_epoch_val': 30,
            'dataset_args': {'dataset_name': 'MNIST', 'root': '~/nta/datasets'},
            'dataset_class': <function torchvisiondataset at 0x7fd310339790>,
            'dist_port': 58481,
            'distributed': False,
            'epochs': 5,
            'epochs_to_validate': [],
            'evaluation_metrics': ['eval_current_task',
                                   'eval_all_visited_tasks'],
            'experiment_class': <class '__main__.ReduceLRContinualLearningExperiment'>,
            'log_level': 'INFO',
            'model_args': {'hidden_sizes': (50, 50, 

Trial name,status,loc
RemoteProcessTrainable_abcf5_00000,RUNNING,


(pid=70081) INFO:ReduceLRContinualLearningExperiment:Execution order: {'aggregate_pre_experiment_results': ['ContinualLearningExperiment.aggregate_pre_experiment_results'],
(pid=70081)  'aggregate_results': ['ContinualLearningExperiment.aggregate_results'],
(pid=70081)  'complexity_loss': [],
(pid=70081)  'create_loaders': ['SupervisedExperiment.create_loaders'],
(pid=70081)  'create_lr_scheduler': ['SupervisedExperiment.create_lr_scheduler'],
(pid=70081)  'create_model': ['SupervisedExperiment.create_model'],
(pid=70081)  'create_optimizer': ['SupervisedExperiment.create_optimizer'],
(pid=70081)  'create_train_dataloader': ['SupervisedExperiment.create_train_dataloader'],
(pid=70081)  'create_train_sampler': ['ContinualLearningExperiment.create_train_sampler'],
(pid=70081)  'create_validation_dataloader': ['SupervisedExperiment.create_validation_dataloader'],
(pid=70081)  'create_validation_sampler': ['ContinualLearningExperiment.create_validation_sampler'],
(pid=70081)  'error_loss':

Trial name,status,loc,iter,total time (s)
RemoteProcessTrainable_abcf5_00000,RUNNING,192.168.0.10:70082,1,7.50353


(pid=70082) INFO:RemoteProcessTrainable:End Iteration Result: {'learning_rate': 0.01}
(pid=70082) 2021-12-13 15:52:25,025	WARNING trainable.py:639 -- Trainable._train is deprecated and will be removed in a future version of Ray. Override Trainable.step instead.
(pid=70081) INFO:ReduceLRContinualLearningExperiment:Setting learning rate to 0.00
(pid=70081) INFO:ReduceLRContinualLearningExperiment:Training...
Result for RemoteProcessTrainable_abcf5_00000:
  date: 2021-12-13_15-52-32
  done: false
  eval_all_visited_tasks__mean_accuracy: 0.48541666666666666
  eval_all_visited_tasks__mean_loss: 2.90626703898112
  eval_all_visited_tasks__total_correct: 466
  eval_all_visited_tasks__total_tested: 960
  eval_current_task__mean_accuracy: 0.9614583333333333
  eval_current_task__mean_loss: 0.14333314895629884
  eval_current_task__total_correct: 923
  eval_current_task__total_tested: 960
  experiment_id: 21b341eacbdb421d92401089d7b49e13
  experiment_tag: '0'
  hostname: Lucas-MacBook-Pro.local
  i

...

Trial name,status,loc,iter,total time (s)
RemoteProcessTrainable_abcf5_00000,RUNNING,192.168.0.10:70082,4,28.7889


(pid=70082) INFO:RemoteProcessTrainable:End Iteration Result: {'learning_rate': 0.001}
(pid=70081) INFO:ReduceLRContinualLearningExperiment:Setting learning rate to 0.00
(pid=70081) INFO:ReduceLRContinualLearningExperiment:Training...
Result for RemoteProcessTrainable_abcf5_00000:
  date: 2021-12-13_15-52-53
  done: true
  eval_all_visited_tasks__mean_accuracy: 0.18333333333333332
  eval_all_visited_tasks__mean_loss: 8.25047353108724
  eval_all_visited_tasks__total_correct: 176
  eval_all_visited_tasks__total_tested: 960
  eval_current_task__mean_accuracy: 0.9552083333333333
  eval_current_task__mean_loss: 0.16957790056864422
  eval_current_task__total_correct: 917
  eval_current_task__total_tested: 960
  experiment_id: 21b341eacbdb421d92401089d7b49e13
  experiment_tag: '0'
  hostname: Lucas-MacBook-Pro.local
  iterations_since_restore: 5
  learning_rate: 0.001
  node_ip: 192.168.0.10
  pid: 70082
  time_since_restore: 35.99036931991577
  time_this_iter_s: 7.201484203338623
  time_tota

Trial name,status,loc,iter,total time (s)
RemoteProcessTrainable_abcf5_00000,TERMINATED,,5,35.9904


Trial name,status,loc,iter,total time (s)
RemoteProcessTrainable_abcf5_00000,TERMINATED,,5,35.9904


**** Trial ended


## Meta-Continual Learning

In [6]:
# ray running in local, if bug, needs to manually shutdown # FIXME
import ray
ray.shutdown()

# networks and datasets under pytorch
import torch
from nupic.research.frameworks.meta_continual_learning.experiments import MetaContinualLearningExperiment
from nupic.research.frameworks.pytorch.datasets import omniglot
from nupic.research.frameworks.meta_continual_learning.models import OMLNetwork
from nupic.research.frameworks.ray.run_with_raytune import run_single_instance

meta_cl_omniglot = dict(
    # experiment
    experiment_class=MetaContinualLearningExperiment,
    distributed=False,
    # dataset
    dataset_class=omniglot,
    dataset_args=dict(root="~/nta/datasets"),
    # model
    model_class=OMLNetwork,
    model_args=dict(num_classes=50),
    fast_params=["adaptation.*"],
    test_train_params=["adaptation.*"],
    # hyperparameters
    batch_size=5,
    num_batches_train=1,
    epochs=2,
    num_tasks_per_epoch=10,
    num_classes=50,
    optimizer_args=dict(lr=1e-4),
    optimizer_class=torch.optim.Adam,
)

# Use one of the existing run functions
run_single_instance(meta_cl_omniglot)


2021-12-13 16:01:17,050	INFO resource_spec.py:223 -- Starting Ray with 10.06 GiB memory available for workers and up to 5.04 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
{'checkpoint_at_end': False,
 'checkpoint_freq': 0,
 'checkpoint_score_attr': None,
 'config': {'batch_size': 5,
            'dataset_args': {'root': '~/nta/datasets'},
            'dataset_class': <function omniglot at 0x7fc3e847ac10>,
            'dist_port': 59205,
            'distributed': False,
            'epochs': 2,
            'experiment_class': <class 'nupic.research.frameworks.meta_continual_learning.experiments.meta_cl_experiment.MetaContinualLearningExperiment'>,
            'fast_params': ['adaptation.*'],
            'log_level': 'INFO',
            'model_args': {'num_classes': 50},
            'model_class': <class 'nupic.research.frameworks.meta_continual_learning.models.OMLNetwork'>,
            'num_batches_train': 1,
            'num_

Trial name,status,loc
RemoteProcessTrainable_f6a9a_00000,RUNNING,


(pid=70573) INFO:MetaContinualLearningExperiment:Execution order: {'adapt': ['MetaContinualLearningExperiment.adapt'],
(pid=70573)  'clone_model': ['MetaContinualLearningExperiment.clone_model'],
(pid=70573)  'complexity_loss': [],
(pid=70573)  'create_loaders': ['MetaContinualLearningExperiment.create_loaders'],
(pid=70573)  'create_lr_scheduler': ['SupervisedExperiment.create_lr_scheduler'],
(pid=70573)  'create_model': ['SupervisedExperiment.create_model'],
(pid=70573)  'create_optimizer': ['SupervisedExperiment.create_optimizer'],
(pid=70573)  'create_replay_dataloader': ['MetaContinualLearningExperiment.create_replay_dataloader'],
(pid=70573)  'create_replay_sampler': ['MetaContinualLearningExperiment.create_replay_sampler'],
(pid=70573)  'create_sampler': ['MetaContinualLearningExperiment.create_sampler'],
(pid=70573)  'create_slow_train_dataloader': ['MetaContinualLearningExperiment.create_slow_train_dataloader'],
(pid=70573)  'create_task_sampler': ['MetaContinualLearningExperi

Trial name,status,loc,acc,loss,iter,total time (s)
RemoteProcessTrainable_f6a9a_00000,RUNNING,192.168.0.10:70569,0.101449,71.3886,1,85.689


(pid=70569) INFO:RemoteProcessTrainable:End Iteration Result: {'total_correct': 7, 'total_tested': 69, 'learning_rate': 0.0001, 'validation_loss': 71.38861846923828, 'validation_accuracy': 0.10144927536231885}
(pid=70569) 2021-12-13 16:03:00,149	WARNING trainable.py:639 -- Trainable._train is deprecated and will be removed in a future version of Ray. Override Trainable.step instead.
Result for RemoteProcessTrainable_f6a9a_00000:
  date: 2021-12-13_16-04-00
  done: true
  experiment_id: 8df281c6cf984ccfa456a6da7ceb4e14
  experiment_tag: '0'
  hostname: Lucas-MacBook-Pro.local
  iterations_since_restore: 2
  learning_rate: 0.0001
  mean_accuracy: 0.10144927536231885
  mean_loss: 28.799962997436523
  neg_mean_loss: -28.799962997436523
  node_ip: 192.168.0.10
  pid: 70569
  time_since_restore: 145.80434799194336
  time_this_iter_s: 60.11535906791687
  time_total_s: 145.80434799194336
  timestamp: 1639440240
  timesteps_since_restore: 0
  total_correct: 7
  total_tested: 69
  training_itera

Trial name,status,loc,acc,loss,iter,total time (s)
RemoteProcessTrainable_f6a9a_00000,TERMINATED,,0.101449,28.8,2,145.804


(pid=70569) INFO:RemoteProcessTrainable:End Iteration Result: {'total_correct': 7, 'total_tested': 69, 'learning_rate': 0.0001, 'validation_loss': 28.799962997436523, 'validation_accuracy': 0.10144927536231885}
(pid=70569) 2021-12-13 16:04:00,278	WARNING trainable.py:821 -- Trainable._stop is deprecated and will be removed in a future version of Ray. Override Trainable.cleanup instead.


Trial name,status,loc,acc,loss,iter,total time (s)
RemoteProcessTrainable_f6a9a_00000,TERMINATED,,0.101449,28.8,2,145.804


**** Trial ended
